In [ ]:
import os, gc, cv2, math, copy, time, random
import pickle

import numpy as np, pandas as pd
from collections import defaultdict

import torch, torch.nn as nn, torch.optim as optim
from torch.optim import lr_scheduler
from torch.cuda import amp
import torch.backends.cudnn as cudnn
import threading


import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import f1_score,roc_auc_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import recall_score

from tqdm.notebook import tqdm

import torch.nn.functional as F
import timm
import ast

from torch.utils.data import Dataset, DataLoader

In [ ]:
seed=8
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.cuda.manual_seed_all(seed)
cudnn.benchmark = False

In [ ]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()

    dataset_size = 0
    running_loss = 0.0

    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    print(f'Epoch:{epoch}')
    
    for step, data in bar:
        ct_b, img_b, c, h, w = data['image'].size()
        data_img = data['image'].reshape(-1, c, h, w)
        data_label = data['label'].reshape(-1,1)
        images = data_img.to(device, dtype=torch.float)
        labels = data_label.to(device, dtype=torch.float)

        
        batch_size = images.size(0)

        with amp.autocast(enabled = True):

            
            outputs = model(images)

            loss = criterion(outputs, labels)

            loss = loss / CONFIG['n_accumulate']

        scaler.scale(loss).backward()

        if (step + 1) % CONFIG['n_accumulate'] == 0:

            scaler.unscale_(optimizer)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()


            if scheduler is not None:
                scheduler.step()
        

        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size

        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()

    return epoch_loss

In [ ]:
@torch.inference_mode()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()

    dataset_size = 0
    running_loss = 0.0

    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    true_y=[]
    pred_y=[]
    for step, data in bar:
        ct_b, img_b, c, h, w = data['image'].size()
        data_img = data['image'].reshape(-1, c, h, w)
        data_label = data['label'].reshape(-1,1)

        images = data_img.to(device, dtype=torch.float)
        labels = data_label.to(device, dtype=torch.float)

        batch_size = images.size(0)

        outputs = model(images)
        loss = criterion(outputs, labels)


        true_y.append(labels.cpu().numpy())
        pred_y.append(torch.sigmoid(outputs).cpu().numpy())

        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size

        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])

    true_y=np.concatenate(true_y)
    pred_y=np.concatenate(pred_y)

    gc.collect()

    true_y=np.array(true_y).reshape(-1,1)
    true_y=np.array(true_y).reshape(-1,img_b)
    true_y=true_y.mean(axis=1)

    pred_y=np.array(pred_y).reshape(-1,1)
    pred_y = torch.nan_to_num(torch.from_numpy(pred_y)).numpy()
    pred_y=np.array(pred_y).reshape(-1,img_b)

    pred_y=pred_y.mean(axis=1)

    assert(true_y.ndim==1)
    assert(pred_y.ndim==1)
    assert (true_y.shape==pred_y.shape)
    
    acc_f1=f1_score(np.array(true_y),np.round(pred_y),average='macro')
    auc_roc=roc_auc_score(np.array(true_y),np.array(pred_y))
    print("acc_f1(mean) : ",round(acc_f1,4),"  auc_roc(mean) : ",round(auc_roc,4))


    return epoch_loss,acc_f1,auc_roc

In [ ]:
def run_training(model, optimizer, scheduler, device, num_epochs):


    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))

    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_loss = np.inf
    best_epoch_auc = 0
    best_epoch_f1 = 0
    history = defaultdict(list)
    for epoch in range(1, num_epochs + 1):
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler, dataloader=train_loader, device=CONFIG['device'], epoch=epoch)
        val_epoch_loss,acc_f1,auc_roc= valid_one_epoch(model, valid_loader, device=CONFIG['device'],epoch=epoch)

        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)


        if val_epoch_loss <= best_epoch_loss:
            print(f"Validation Loss Improved ({best_epoch_loss} ---> {val_epoch_loss})")
            best_epoch_loss = val_epoch_loss

            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f'{bin_save_path}/loss_ECA/'+job_name
            os.makedirs(f'{bin_save_path}/loss_ECA/', exist_ok=True)
            torch.save(model.module.state_dict(), PATH)
            print(f"Model Saved")

        if auc_roc >= best_epoch_auc:
            print(f"Validation Auc Improved ({best_epoch_auc} ---> {auc_roc})")
            best_epoch_auc = auc_roc

            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f'{bin_save_path}/auc_roc_ECA/'+job_name
            os.makedirs(f'{bin_save_path}/auc_roc_ECA/', exist_ok=True)
            torch.save(model.module.state_dict(), PATH)
            print(f"Model Saved")

        if acc_f1 >= best_epoch_f1:
            print(f"Validation f1 Improved ({best_epoch_f1} ---> {acc_f1})")
            best_epoch_f1 = acc_f1

            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f'{bin_save_path}/f1_ECA/'+job_name
            os.makedirs(f'{bin_save_path}/f1_ECA/', exist_ok=True)
            torch.save(model.module.state_dict(), PATH)
            print(f"Model Saved")

    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss: {:.4f}".format(best_epoch_loss))
    return model, history

In [ ]:
CONFIG = {
        "epochs":  100, 
        "img_size": 384,
        "valid_batch_size": 8,
        "learning_rate": 0.0001,

        "weight_decay": 0.0005, 

        "n_accumulate": 1, 
        "device": torch.device("cuda"),
        }

In [ ]:
data_transforms = {
"train": A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.ShiftScaleRotate(shift_limit=0.2, 
                       scale_limit=0.2, 
                       rotate_limit=30, 
                       p=0.5),
    A.HueSaturationValue(
            hue_shift_limit=0.2, 
            sat_shift_limit=0.2, 
            val_shift_limit=0.2, 
            p=0.5 
        ),
    A.RandomBrightnessContrast(
            brightness_limit=(-0.2,0.2), #0.2
            contrast_limit=(-0.2, 0.2),  #0.2
            p=0.5 
        ),
    A.dropout.coarse_dropout.CoarseDropout(p=0.2),
    A.Normalize(),
    ToTensorV2()], p=1.),

"valid": A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),

    A.Normalize(),
    ToTensorV2()], p=1.)
}

In [ ]:
files_dir='/mnt/challenge_preprocessing/files/filtered'

challenge1_train = pd.read_csv(os.path.join(files_dir,'filtered-la-challenge1_train_path_range_label.csv'))
challenge1_valid = pd.read_csv(os.path.join(files_dir,'filtered-la-challenge1_valid_path_range_label.csv'))

challenge2_train = pd.read_csv(os.path.join(files_dir,'filtered-la-challenge2_train_path_range_label.csv'))
challenge2_valid = pd.read_csv(os.path.join(files_dir,'filtered-la-challenge2_valid_path_range_label.csv'))

In [ ]:
# challenge_valid = pd.concat([challenge1_valid, challenge2_valid], axis=0)
challenge_train=  pd.concat([challenge1_train, challenge2_train, challenge2_valid], axis=0)
challenge_train=  challenge_train.sample(frac=1).reset_index(drop=True)

#challenge_train=challenge1_train
challenge_valid=challenge1_valid

In [ ]:
class ChallengeDataset(Dataset):
    def __init__(self, df, transforms=None, batch_size=16):
        self.img_paths = df['Path'].tolist()
        self.labels = df['Label'].values
        self.transforms = transforms
        
        self.ranges = df['Range'].apply(ast.literal_eval).tolist()
        self.batch_size = batch_size

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        label = self.labels[idx]
        range_list = self.ranges[idx]
        
        if '/challenge1' in img_path:
            img_path = img_path.replace('preprocessed', 'OriginalDatauncompressed')
        else:
            img_path = img_path.replace('preprocessed', 'dataset')
         
        if ('neg' in img_path and label == 1) or ('pos' in img_path and label == 0) or ('/co' in img_path and label == 0) or ('/no' in img_path and label == 1):
            print(img_path, label)
        
        file_paths = list(range(range_list[0], range_list[1]))
        
        if len(file_paths) > self.batch_size:
            sampled_paths = np.random.choice(file_paths, size=self.batch_size, replace=False)       
        else:
            sampled_paths = file_paths
        sampled_paths = np.sort(sampled_paths)
        
        images = torch.empty((self.batch_size, 3, 384, 384))
        labels = torch.empty((self.batch_size,1))    
        
        for i, path in enumerate(sampled_paths):
            img = cv2.imread(os.path.join(img_path, f"{path}.jpg"))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            img = self.transforms(image=img)['image']
            images[i] = img[:]
            labels[i] = label
            

        return {
            'image': images,
            'label': torch.tensor(labels, dtype=torch.long)
        }

In [ ]:
train_dataset=ChallengeDataset(challenge_train,data_transforms['train'],12)
valid_dataset=ChallengeDataset(challenge_valid,data_transforms['valid'],12)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=14,  shuffle=False, pin_memory=True, drop_last=True)
valid_loader = DataLoader(valid_dataset, batch_size=14,  shuffle=False)

In [ ]:
class eca_nfnet_l0(nn.Module):
    def __init__(self):
        super(eca_nfnet_l0, self).__init__()

        self.model = timm.create_model("hf_hub:timm/eca_nfnet_l0", pretrained=True)
        self.classifier = nn.Linear(self.model.head.fc.in_features, 1, bias=True)
        
        self.attention = nn.Conv2d(2, 1, kernel_size=1, bias=True)
        
        layer_name = 'final_conv'
        
        self.features = {}
        
        self.model.final_act.register_forward_hook(self.get_features)

    def set_features(self, features):
        self.features = features

    def get_features(self, module, input, output):
        self.features[threading.get_ident()] = output

    def getAttFeats(self, att_map, features):
        features = 0.5 * features + 0.5 * (att_map * features)
        return features

    def forward(self, x):
        outputs = {}
        
        dummy = self.model(x)
        
        features = self.features[threading.get_ident()]
        fg_att = self.attention(torch.cat((torch.mean(features, dim=1).unsqueeze(1), torch.max(features, dim=1)[0].unsqueeze(1)), dim=1))
        fg_att = torch.sigmoid(fg_att)
        features = self.getAttFeats(fg_att, features)
        
        out = F.adaptive_avg_pool2d(features, (1, 1))
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        
        outputs['logits'] = out
        outputs['feat'] = features
        
        return out
    
    
bin_save_path = "/mnt/saved_models/14batch_eachof12_combing_challenge1_2training_challene2validation"
job_name = f"epoch:{CONFIG['epochs']}_ECA_Attention_{CONFIG['img_size']}"
model = eca_nfnet_l0()
print(model)

In [ ]:
criterion=nn.BCEWithLogitsLoss()
model = nn.DataParallel(model)#, device_ids=[0,1,2,3])

model = model.to(CONFIG['device'])

scaler = amp.GradScaler()

print("="*10, "*model* setting", "="*10)
optimizer = optim.Adam(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])

print("="*10, "Start Train", "="*10)
model, history= run_training(model, optimizer,None, device=CONFIG['device'], num_epochs=CONFIG['epochs'])